In [1]:
import pandas as pd
import requests
import os

# The following is to allow all columns to be displayed in the Jupyter notebook
# instead of a limited number
from IPython.display import display
pd.options.display.max_columns = None

In [2]:
# Years 2004 - 2018 have the most consistent structure to their data files
for year in range(2004, 2019):

    year = str(year)
    shortyear = year[-2:]
    baseurl = 'https://www2.census.gov/programs-surveys/saipe/datasets/'
    url = f'{baseurl}/{year}/{year}-state-and-county/est{shortyear}all.xls'
    r = requests.get(url, allow_redirects=False)
    
    # Save the contents to a .xls file, note that the contents 
    # are returned in byte form so we need to use mode='wb'
    output_file = os.path.split(url)[1]
    outfile = open(os.path.join('data', output_file), mode='wb')
    outfile.write(r.content)
    outfile.close() # remember to close the connection to the file

In [3]:
dict_of_spreadsheets = {}

for year in range(2004, 2019):
    if year < 2005:
        header_row_num = 1
    elif year < 2013:
        header_row_num = 2
    else:
        header_row_num = 3
    
    shortyear = str(year)[-2:]
    
    key = f'df_{year}'
    
    val = pd.read_excel('data/est' + shortyear + 'all.xls',
                            header=header_row_num)
    
    val['Year'] = [year]*val.shape[0]
    
    dict_of_spreadsheets.update({key:val})
    
    print(f'Year = {year},\tDimensions of DataFrame = {val.shape}')

Year = 2004,	Dimensions of DataFrame = (3193, 32)
Year = 2005,	Dimensions of DataFrame = (3196, 32)
Year = 2006,	Dimensions of DataFrame = (3196, 32)
Year = 2007,	Dimensions of DataFrame = (3196, 32)
Year = 2008,	Dimensions of DataFrame = (3197, 32)
Year = 2009,	Dimensions of DataFrame = (3198, 32)
Year = 2010,	Dimensions of DataFrame = (3198, 32)
Year = 2011,	Dimensions of DataFrame = (3198, 32)
Year = 2012,	Dimensions of DataFrame = (3195, 32)
Year = 2013,	Dimensions of DataFrame = (3195, 32)
Year = 2014,	Dimensions of DataFrame = (3194, 32)
Year = 2015,	Dimensions of DataFrame = (3194, 32)
Year = 2016,	Dimensions of DataFrame = (3194, 32)
Year = 2017,	Dimensions of DataFrame = (3194, 32)
Year = 2018,	Dimensions of DataFrame = (3194, 32)


In [4]:
# Fixing the column names so that all dataframes have the same column names

for year in range(2012,2019):
    
    new_col_names = [oldname.replace(',','') for oldname in list(dict_of_spreadsheets['df_2013'].columns)]
    dict_of_spreadsheets[f'df_{year}'].columns = new_col_names

    
    
fix_col_names = {'Postal':'Postal Code', 'County FIPS':'County FIPS Code', 'State FIPS':'State FIPS Code',
                'Poverty Estimate Under Age 18':'Poverty Estimate Age 0-17',
                'Poverty Estimate Ages 0-4':'Poverty Estimate Age 0-4',
                'Poverty Estimate Ages 5-17':'Poverty Estimate Age 5-17 in Families',
                'Poverty Percent Under Age 18':'Poverty Percent Age 0-17',
                'Poverty Percent Ages 0-4':'Poverty Percent Age 0-4',
                'Poverty Percent Ages 5-17':'Poverty Percent Age 5-17 in Families'}

for year in range(2004,2012):
    dict_of_spreadsheets[f'df_{year}'].rename(columns=fix_col_names, inplace=True)

In [5]:
dat = pd.concat([dict_of_spreadsheets[key] for key in dict_of_spreadsheets.keys()]).reset_index(drop=True)

# Get rid of rows that are commentary from Excel
dat = dat[dat['Postal Code'].notna()].reset_index(drop=True)

# Rearrange the columns so that 'Year' is first
cols = [dat.columns[-1]]
cols.extend(dat.columns[0:-1])

dat = dat[cols]

In [6]:
dat

,Year,State FIPS Code,County FIPS Code,Postal Code,Name,Poverty Estimate All Ages,90% CI Lower Bound,90% CI Upper Bound,Poverty Percent All Ages,90% CI Lower Bound.1,90% CI Upper Bound.1,Poverty Estimate Age 0-17,90% CI Lower Bound.2,90% CI Upper Bound.2,Poverty Percent Age 0-17,90% CI Lower Bound.3,90% CI Upper Bound.3,Poverty Estimate Age 5-17 in Families,90% CI Lower Bound.4,90% CI Upper Bound.4,Poverty Percent Age 5-17 in Families,90% CI Lower Bound.5,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,Poverty Estimate Age 0-4,90% CI Lower Bound.7,90% CI Upper Bound.7,Poverty Percent Age 0-4,90% CI Lower Bound.8,90% CI Upper Bound.8
0,2004,0,0.0,US,United States,37039804,36359662,37719946,12.7,12.5,13,13041492,12694086,13388898,17.8,17.3,18.3,8430886,8142299,8719473,16.2,15.6,16.7,44334,44012,44656,4.12372e+06,3.91791e+06,4.32952e+06,20.5,19.5,21.5
1,2004,1,0.0,AL,Alabama,717483,675992,758974,16.1,15.1,17,245017,226152,263881,22.6,20.9,24.4,160787,143834,177740,20.7,18.5,22.9,37062,35614,38510,77330,68732,85928,26.2,23.3,29.1
2,2004,1,1.0,AL,Autauga County,5627,4340,6914,11.6,8.9,14.3,2041,1518,2565,16.3,12.1,20.5,1328,938,1718,14.4,10.2,18.6,45379,42625,48310,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,1,3.0,AL,Baldwin County,16089,12434,19744,10,7.7,12.3,5415,4019,6812,14.9,11.1,18.8,3521,2484,4558,13.3,9.4,17.3,42227,39726,44886,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,1,5.0,AL,Barbour County,6109,4685,7533,23.9,18.3,29.4,2024,1494,2555,31.1,22.9,39.2,1329,931,1727,28,19.6,36.4,26495,24830,28272,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47906,2018,56,37.0,WY,Sweetwater County,3540,2804,4276,8.4,6.7,10.1,1099,802,1396,10,7.3,12.7,697,479,915,8.5,5.8,11.2,73315,67141,79489,.,.,.,.,.,.
47907,2018,56,39.0,WY,Teton County,1443,1137,1749,6.3,5,7.6,278,202,354,6.7,4.9,8.5,180,126,234,5.9,4.1,7.7,99087,88142,110032,.,.,.,.,.,.
47908,2018,56,41.0,WY,Uinta County,2010,1585,2435,10,7.9,12.1,696,494,898,12.1,8.6,15.6,454,306,602,10.6,7.2,14,63401,56812,69990,.,.,.,.,.,.
47909,2018,56,43.0,WY,Washakie County,918,700,1136,11.9,9.1,14.7,277,197,357,16.1,11.4,20.8,191,127,255,14.5,9.7,19.3,55190,49263,61117,.,.,.,.,.,.


In [7]:
# Save to one large csv file
dat.to_csv('data/estUSall_2004-2018.csv', index=False)